In [ ]:
import numpy as np
import scanpy as sc
import seaborn as sns
import os
import pandas as pd
import matplotlib.pyplot as plt
import warnings
import sys
import pickle as pkl
path_helper = ["C:\\","Users","vfriedrich","projects","monkey_IZI","git_documentation","scRNAseq_cross_species_primate_human","analysis","helper"]
sys.path.append(os.path.join(*path_helper))
import helperVDF as h
#import decoupler
print(sys.executable)
import gseapy
from gseapy.plot import barplot, dotplot
warnings.filterwarnings("ignore")
import re

In [ ]:
pre = "MH122"
drive = 'F'
base_model_path,base_table_path,base_plots_path,base_anndata_objects = h.return_local_paths(drive = drive,
                                                                                            pre = pre,
                                                                                            add_path = True)
def remove_go(term):
    # Use regular expression to find and replace GO terms, ensuring the string is treated as a raw string
    new_term = re.sub(r'\s*\(GO:\d+\)', '', term)  # Corrected regex finds the GO term pattern and removes it
    return new_term.strip() 



In [ ]:
_,base_table_path_MH68,_,_ = h.return_local_paths(drive = drive,pre = 'MH68',add_path = True)
df_feature_importance_VAE = pd.read_csv(os.path.join(base_table_path_MH68,'MH68df_pert_all.csv'),index_col=0)
celltypes = ['B', 'CD14 Mono', 'CD16 Mono', 'CD4 T', 'CD8 T', 'MAIT','NK+Proliferating']

all_pathways_VAE = []
for celltype in celltypes:
    celltypes_filtered = df_feature_importance_VAE[df_feature_importance_VAE['celltype'] == celltype] 
    
    celltypes_filtered_sorted = celltypes_filtered.sort_values('mean_perturbation_effect', ascending=False)
    
    top_genes = list(celltypes_filtered_sorted.head(200)['mean_perturbation_effect'].index)
    
    celltypes_filtered_sorted['human_gene_name'] = [gene.split('--')[0] for gene in celltypes_filtered_sorted['mean_perturbation_effect'].index] 
    
    top_genes = list(celltypes_filtered_sorted.head(200)['human_gene_name'].value_counts().index)
    
    enrichment = gseapy.enrichr(gene_list=top_genes,
                 gene_sets=['Reactome_Pathways_2024','WikiPathway_2023_Human','GO_Biological_Process_2023','KEGG_2021_Human','KEGG_2016'],
                 organism='Human', 
                 cutoff=1
                 )
    
    df = enrichment.results
    
    df['celltype'] = celltype
    df.to_csv(os.path.join(base_table_path,pre + '_df_pathways_VAE_' + str(celltype) + '.csv'))
    filtered_df = df[df['Adjusted P-value'] < 0.05]
    filtered_df['Foreground Count'] = filtered_df['Overlap'].apply(lambda x: int(x.split('/')[0]))
    filtered_df = filtered_df[filtered_df['Foreground Count'] >= 2]
    filtered_df = filtered_df.sort_values(by='Odds Ratio', ascending=False)
    filtered_df['Log Odds Ratio'] = np.log(filtered_df['Odds Ratio'])
    filtered_df['short_term'] = filtered_df['Term'].apply(remove_go)
    filtered_df.to_csv(os.path.join(base_table_path,pre + '_filtered_df_pathways_VAE_' + str(celltype) + '.csv'))
    all_pathways_VAE.append(filtered_df)
df_all_pathways_VAE = pd.concat(all_pathways_VAE,ignore_index=True)
df_all_pathways_VAE.to_csv(os.path.join(base_table_path,pre + '_df_all_pathways_VAE.csv'))

save session

In [ ]:
np.max(df['P-value'])

In [ ]:
base_package_version_path = h.return_package_version_local_path(drive=drive)
h.save_package_versions(base_package_version_path,pre,do_print = True)
h.print_main_versions()